In [6]:
import os
import keras
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers.merge import concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import  Input, Dense, Conv2D, Dropout, Flatten, MaxPooling2D, LSTM
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

In [7]:
def load_cnn_model(model_path='./result/test'):
    with open(os.path.join(model_path, 'model_architecture.json'), 'r') as f:
        model = model_from_json(f.read())

    model.load_weights(os.path.join(model_path, 'weights-42-0.90.hdf5'))
    
    return model

In [8]:
def load_rnn_model(model_path='./result/rnn_model'):
    with open(os.path.join(model_path, 'model_architecture.json'), 'r') as f:
        model = model_from_json(f.read())

    model.load_weights(os.path.join(model_path, 'weights-improvement-94-0.15.hdf5'))
    
    return model

In [9]:
def load_data(dataset_path='./npy_files'):
    X = np.load(os.path.join(dataset_path,'X.npy'))
    y = np.load(os.path.join(dataset_path,'y.npy'))
    folds = np.load(os.path.join(dataset_path,'folds.npy'))
    feature_class = np.load(os.path.join(dataset_path, 'feature_class.npy'))
    facs = np.load(os.path.join(dataset_path,'np_facs.npy'))

    # rescale [0,255] -> [0,2]    
    X = X.astype('float32')/255*2

    # one-hot encode the labels
    num_classes = len(np.unique(y))
    y = keras.utils.to_categorical(y, num_classes)

    return X, y, folds, feature_class, facs

In [10]:
def make_facs_y(facs):
    facs_y = []
    for inst in facs:
        temp = list(inst)[:-1]
        temp.append(list(np.zeros(64)))
        facs_y.append(temp)
    facs_y = np.array(facs_y)
    
    return facs_y

In [11]:
def make_cv_folds(X, y, feature_class, facs, folds, fold_num):
    X_train = []
    y_train = []
    feature_class_train = []
    facs_train = []
    
    X_valid = []
    y_valid = []
    feature_class_valid = []
    facs_valid = []
    
    for index, fold in enumerate(folds):
        if fold == fold_num:
            X_valid.append(X[index])
            y_valid.append(y[index])
            feature_class_valid.append(feature_class[index])
            facs_valid.append(facs[index])
        else:
            X_train.append(X[index])
            y_train.append(y[index])
            feature_class_train.append(feature_class[index])
            facs_train.append(facs[index])
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_valid = np.array(X_valid)
    y_valid = np.array(y_valid)
    
    feature_class_train = np.array(feature_class_train)
    feature_class_valid = np.array(feature_class_valid)

    facs_train = np.array(facs_train)
    facs_valid = np.array(facs_valid)
    facs_y_train = make_facs_y(facs_train)
    facs_y_valid = make_facs_y(facs_valid)

    return (X_train, y_train, 
            X_valid, y_valid,
            feature_class_train, feature_class_valid,
            facs_train, facs_valid,
            facs_y_train, facs_y_valid
           )

In [12]:
cnn_model = load_cnn_model()
rnn_model = load_rnn_model()

In [13]:
X, y, folds, feature_class, facs = load_data('./save_data/npy_files/')

X_train, y_train, X_valid, y_valid, feature_class_train, feature_class_valid,facs_train, facs_valid,facs_y_train, facs_y_valid = make_cv_folds(X, y, feature_class,
                                            facs, folds, 0)

In [17]:
facs_prob = rnn_model.predict([facs_valid,feature_class_valid])

In [18]:
facs_classes = facs_prob.argmax(axis=2)

In [19]:
facs_classes.shape

(132, 15)

In [24]:
facs[3]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0